https://womantalks.ru/forum/65-puteshestviya/

In [14]:
# pip install bs

In [15]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from tqdm.notebook import tqdm

df = pd.DataFrame(columns=['text'])
df

,text


In [16]:
url = 'https://womantalks.ru/forum/65-puteshestviya/'

In [17]:
response = requests.get(url)
print(response)

<Response [200]>


In [18]:
html = response.text

In [19]:
# bs = BeautifulSoup(response.text,"lxml")
# print(bs)

In [20]:
import re

In [21]:
def get_links(html, link):
    soup = BeautifulSoup(html, features="html.parser")
    links = []
    tds = soup.find_all('li', {'class':'ipsPagination_page'})
    for td in tds:
        p = td.find_all('a')
        for q in p:
            if link in q['href']:
                links.append(q['href'])
    return sorted(list(set(links)))
get_links(get_html(url), url)

[]

In [26]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import requests
import json

def get_html(url):
    response = requests.get(url)
    return response.text

def get_all_links(html):
    soup = BeautifulSoup(html, features="html.parser")
    links = []
    tds = soup.find_all('h4', {'class':'ipsDataItem_title ipsType_large ipsType_break'})
    for td in tds:
        p = td.find_all('a')
        for q in p:
            links.append(q['href'])
    return links

# def get_all_links_on_page(html, link):
#     soup = BeautifulSoup(html, features="html.parser")
#     tds = soup.find_all('tr')
#     for td in tds:
#         i = td.find('td', {'class':'td-block'})
#         if i != None:
#             return(i.find_all('a')[-1].text)
    
def get_links(html, link):
    soup = BeautifulSoup(html, features="html.parser")
    links = []
    tds = soup.find_all('li', {'class':'ipsPagination_page'})
    for td in tds:
        p = td.find_all('a')
        for q in p:
            if link in q['href']:
                links.append(q['href'])
    if len(links) == 0:
        return [link]
    return sorted(list(set(links)))

def get_all_links_one_theme(html):
    soup = BeautifulSoup(html, features="html.parser")
    links = []
    tds = soup.find_all('h4', {'class':'ipsDataItem_title ipsType_break'})
    for td in tds:
        p = td.find_all('a')
        for q in p:
            if 'topic' in q['href']:
                links.append(q['href'])
    return links

def get_page_data(html):
    soup = BeautifulSoup(urlopen(str(html.encode('utf-8'))[2:-1]), features="html.parser")

    # user
    tds = soup.find_all('strong', 'itemprop'=="name")
    users = []
    for td in tds:
        if 'Изменено' not in td.text:
            users.append(td.text)
#     print(users)


    # time
    tds = soup.find_all('p', {'class':"ipsType_reset"})
    times = []
    for td in tds:
        if 'Опубликовано' in td.text:
            times.append(td.text)
#     print(times)


    # text
    tds = soup.find_all('div', {'class':'ipsType_normal ipsType_richText ipsContained'})
    tex = []
    for td in tds: 
        if td.find('blockquote') != None:
            p = td.find_all('p')
            tex.append(p[-1].text)
        else:
            tex.append(td.text)
#     print(tex)
    

    # quote
    tds = soup.find_all('div', {'class':'ipsType_normal ipsType_richText ipsContained'})
    temp = []
    for td in tds: 
        if td.find('blockquote') != None:
            p = td.find('div', {'class':"ipsQuote_contents"})
            if p is not None:
                temp.append(p.text)
        else:
            temp.append('')
#     print(temp)

    data = ({'user': users,                    
             'time': times,      
             'text': tex,            
             'quote': temp,          
             'topic': ['Travel'] * len(users)
            })          
    return users, times, tex, temp, ['Travel'] * len(users)#data

def page_content(all_links, table):
#     for link in tqdm(all_links):
#     links = get_links(get_html(all_links), all_links)
    i = 0
    for l in tqdm(all_links):
        all_links_one_theme = get_all_links_one_theme(get_html(l))
        if i == 0:
            all_links_one_theme.append('https://womantalks.ru/topic/1517-turizm/')
            all_links_one_theme.append('https://womantalks.ru/topic/102-veloturizm/')
            i=1
        
        for t in tqdm(all_links_one_theme):
            print(t)
            page = get_links(get_html(t), t)
            for p in page:
#                 print(p)
                us, ti, te, tem, l = get_page_data(p)
                for i in range(len(us)):
                    if i > 5:
                        break
                    users.append(us[i])
                    times.append(ti[i])
                    tex.append(te[i])
                    temp.append(tem[i])
                    labels.append(l[i])
#                 table.append(data)

table = pd.DataFrame(columns={'users', 'times', 'text', 'quote', 'topic'})
users = []
times = []
tex = []
temp = []
labels = []
    
url = 'https://womantalks.ru/forum/65-puteshestviya/'

all_links = get_links(get_html(url), url)
print(all_links)
page_content(all_links, table)

table.loc[:, 'users'] = users
table.loc[:, 'times'] = times
table.loc[:, 'text'] = tex
table.loc[:, 'quote'] = temp
table.loc[:, 'topic'] = labels

['https://womantalks.ru/forum/65-puteshestviya/']


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

https://womantalks.ru/topic/1517-turizm/
https://womantalks.ru/topic/102-veloturizm/


In [27]:
times

[]

In [44]:
pd.DataFrame(table).to_csv('Travel_woman_talks.csv', index=False)

In [45]:
D = pd.read_csv('Travel_woman_talks.csv')
D

,user,time,text,quote,topic
0,"['\nГость', '\nJasmiine', '\nGolubka']","['\nОпубликовано: 21 мар 2019\n', '\nОпубликов...","['\n\n\tПоделитесь, пожалуйста кто где путешес...","['', '', '']","['Travel', 'Travel', 'Travel']"
1,"['\nДарина', '\nМиленькая)', '\nЛаванда', '\nL...","['\nОпубликовано: 18 апр 2016\n', '\nОпубликов...",['\n\n\tКуда вы хотите поехать в отпуск? Я бы ...,"['', '', '', '', '\n\n\t\t\tМы пока еще не выб...","['Travel', 'Travel', 'Travel', 'Travel', 'Trav..."
2,"['\nМарта', '\nXyhuvupu', '\nOlgaOlga_5', '\nT...","['\nОпубликовано: 13 мар 2017\n', '\nОпубликов...",['\n\n\tНам как вариант предложили купить недв...,"['', '', '\n\n\t\t\tа я по стране хочу поездит...","['Travel', 'Travel', 'Travel', 'Travel', 'Trav..."
3,"['\nEkaterinaN', '\nLolaa', '\nAlivia', '\nКюш...","['\nОпубликовано: 23 апр 2019\n', '\nОпубликов...","['\n\n\tГреция по дороже, но гораздо лучше. Бы...","['', '', '', '', '']","['Travel', 'Travel', 'Travel', 'Travel', 'Trav..."
4,"['\nGolubka', '\nbaranets20-20', '\nGolubka', ...","['\nОпубликовано: 30 янв 2021\n', '\nОпубликов...","['\n\n\tЧто лучше, ехать со своей карточкой ил...","['', '', '', '']","['Travel', 'Travel', 'Travel', 'Travel']"
5,"['\nAlexa', '\nNissa', '\nМикра', '\nAlexa', '...","['\nОпубликовано: 2 мар 2016\n', '\nОпубликова...",['\n\n\tЧасто встречаю приверженников самостоя...,"['', '', '\n\n\t\t\tЧасто встречаю приверженни...","['Travel', 'Travel', 'Travel', 'Travel', 'Trav..."
6,"['\nikari', '\ncorenad', '\nKetaryKa']","['\nОпубликовано: 3 авг 2020\n', '\nОпубликова...","['\n\n\t\xa0Как сейчас дела с развлечениями, ч...","['', '', '']","['Travel', 'Travel', 'Travel']"
7,"['\nKatosha', '\nАнтонина89', '\nГалина Марков...","['\nОпубликовано: 12 фев 2020\n', '\nОпубликов...",['\n\n\tДавайте поговорим об Израиле. Кто был?...,"['', '', '', '', '']","['Travel', 'Travel', 'Travel', 'Travel', 'Trav..."
8,"['\nБьянка-Янка', '\nМайская', '\nAleksandra11...","['\nОпубликовано: 28 май 2016\n', '\nОпубликов...",['\n\n\tКак предпочитаете отдыхать - валяться ...,"['', '', '', '', '', '', '', '', '', '', '', '...","['Travel', 'Travel', 'Travel', 'Travel', 'Trav..."
9,['\nVenezian'],['\nОпубликовано: 17 дек 2019\n'],"['\n\n\tВсем привет, очередной вопрос про пода...",[''],['Travel']


In [46]:
count = 0
for i in range(len(D)):
    count += len(D['user'][i].split())
print('Количество записей с одного сайта по одной теме:', count)

Количество записей с одного сайта по одной теме: 415
